In [11]:
import pandas as pd
from sqlalchemy import create_engine,text
import joblib
import os
from sklearn.metrics import classification_report

# 2. Loading the Model and Datasets

In [ ]:

model = joblib.load('sleep_quality_classifier.pkl')
best_features = joblib.load('best_features.pkl')
scaler = joblib.load('scaler.pkl')
X_val = pd.read_csv('datasets/X_val.csv')
y_val = pd.read_csv('datasets/y_val.csv')

#  Database Connection 

In [ ]:

host = r'127.0.0.1'  
db = r'MSDS610'
user = r'postgres' 
pw = r'12345'  
port = r'5432'  

db_conn = create_engine("postgresql://{}:{}@{}:{}/{}".format(user, pw, host, port, db))

#  Data Preprocessing 

In [ ]:

X_val_subset = X_val[best_features]
X_val_scaled = scaler.transform(X_val_subset)

# Generating Predictions for the Validation Set

In [ ]:

y_val_pred = model.predict(X_val_scaled)

In [14]:
y_val_pred

array([0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1],
      dtype=int64)

# Storing Predictions in Database

In [ ]:

predictions_df = pd.DataFrame({'ID': X_val.index, 'Predicted_Sleep_Quality': y_val_pred})

In [16]:
predictions_df.to_sql('sleep_quality_predictions', db_conn, schema='analytics', if_exists='replace', index=False)

131

Here i am storing the data in dataframe and writing the dataframe to the table

# summary

This notebook focuses on generating predictions from a pre-trained sleep quality classification model and storing these predictions in a PostgreSQL database.  The trained model along with the best features and scaler are loaded using joblib.  The validation dataset (X_val) is also loaded and the relevant features are selected and scaled using the loaded scaler.  Predictions are then generated using the loaded model.  A Pandas DataFrame is created to store these predictions associating each prediction with an ID (taken from the index of X_val).  Finally this DataFrame is written to a PostgreSQL database table named sleep_quality_predictions within the analytics schema.  The if_exists='replace' argument ensures that any existing table with the same name is replaced. The index is not saved to the database. The resulting table contains the IDs and their corresponding predicted sleep quality values (0 or 1). There are 131 rows in the resulting table.
